In [11]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils

#We’ll also import the Thread  class so we can play our alarm in a separate 
#thread from the main thread to ensure our script doesn’t pause execution 
#while the alarm sounds.
from threading import Thread

import numpy as np

#In order to actually play our WAV/MP3 alarm, we need the playsound library,
#a pure Python, cross-platform implementation for playing simple sounds.
import playsound

import time
import dlib
import cv2
from collections import OrderedDict

In [12]:
path = '/home/tejas/Desktop/CV/Loud_Alarm_Clock_Buzzer-Muk1984-493547174.mp3'

def sound_alarm(path):
    # play an alarm sound
    playsound.playsound(sound = path)

In [13]:
def eye_aspect_ratio(eye):
    # compute the dist between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1],eye[5])
    B = dist.euclidean(eye[2], eye[4])
 
    # compute the euclidean dist between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
 
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
 
    return ear

In [14]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)
 
    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
 
    # return the list of (x, y)-coordinates
    return coords

In [15]:
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold for to set off the
# alarm
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 48


# initialize the frame counter and a boolean used to
# indicate if the alarm is going off
COUNTER = 0
ALARM_ON = False

In [16]:
facial_features = OrderedDict([
    ("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 35)),
    ("jaw", (0, 17))
])

In [17]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/home/tejas/Desktop/CV/shape_predictor_68_face_landmarks.dat')

In [18]:
# grab the indexes of the facial landmarks for the left and
# right eye
(lstart, lend) = facial_features["left_eye"]
(rstart, rend) = facial_features["right_eye"]

In [21]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #detect faces in the grayscale frame
    #1 for images 0 for video feed
    rects = detector(gray,0)
    
    #loop over the face detections
    for rect in rects:
        #determine the facial landmark for the face region, then convert the
        #facial landmark (x,y) coordinates to numpy array
        shape = predictor(gray,rect)
        shape = shape_to_np(shape)
        
        # extract the left and right eye coords., then use the
        # coords. to compute the EAR for both eyes
        left_eye = shape[lstart:lend]
        right_eye = shape[rstart:rend]
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        
        #avg. the EAR together for both eyes
        ear = (left_ear + right_ear)/2.0
        
        #compute the convex hull for the left and right eye, then visualise
        # each of the eyes
        left_eye_hull = cv2.convexHull(left_eye)
        right_eye_hull = cv2.convexHull(right_eye)
        cv2.drawContours(frame,[left_eye_hull],-1,(0,255,0),1)
        cv2.drawContours(frame,[right_eye_hull],-1,(0,255,0),1)
        
        if ear < EYE_AR_THRESH:
            COUNTER += 1
            
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                if not ALARM_ON:
                    #if alarm not on then turn it on
                    ALARM_ON = True
                    
                    if path != "":
                        t = Thread(target=sound_alarm,args=(path,))
                        t.deamon = True
                        t.start()
                        
                cv2.putText(frame,"DROWSINESS ALERT!",(10,30),
                            cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
                
        
        
        #otherwise EAR not below threshold then reset alarm
        else:
            COUNTER = 0
            ALARM_ON = False
            
        # draw the computed eye aspect ratio on the frame to help
        # with debugging and setting the correct eye aspect ratio
        # thresholds and frame counters
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
TOTAL = 0
COUNTER = 0

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/home/tejas/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/tejas/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-12-c7e54432bb95>", line 5, in sound_alarm
    playsound.playsound(sound = path)
  File "/home/tejas/tensorflow/lib/python2.7/site-packages/playsound.py", line 91, in _playsoundNix
    import gi
ImportError: No module named gi

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/home/tejas/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/tejas/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-12-c7e54432bb95>", line 5, in sound_alarm
    playsound.playsound(sound = path)
  File "/home/tejas/tensorflow/lib/python2.7/site-packages/